In [1]:
%%capture
!pip install pygame
!pip install pygad

!git clone https://github.com/karinemiras/evoman_framework.git tmp
!cp -r /kaggle/working/tmp/* /kaggle/working/
!rm -R /kaggle/working/tmp

In [2]:
import sys, os
import numpy as np
import pandas as pd

import random

from evoman.environment import Environment
from evoman.controller import Controller
from demo_controller import player_controller
from deap import base, creator, tools, algorithms
import multiprocessing
import optuna

import seaborn as sns
import matplotlib.pyplot as plt

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
def parameter_count(hidden_neurons):
    if hidden_neurons>0:
        n_w = (20*hidden_neurons) + (hidden_neurons*5)
        n_b = hidden_neurons + 5
        return n_w + n_b
    else:
        return (20*5)+5
    
def fitness_function(solution):
    sum_gain = 0
    for enemy in range(1,9):
        env.update_parameter("enemies",[enemy])
        res = env.play(np.array(solution))
        gain = res[1]-res[2]
        sum_gain += gain
    return (sum_gain,)

"""
def fitness_function(solution):
    defeated = 0
    for enemy in range(1,9):
        env.update_parameter("enemies",[enemy])
        res = env.play(np.array(solution))
        if res[2]<=0:
            defeated += 1
    return (defeated,)
"""

'\ndef fitness_function(solution):\n    defeated = 0\n    for enemy in range(1,9):\n        env.update_parameter("enemies",[enemy])\n        res = env.play(np.array(solution))\n        if res[2]<=0:\n            defeated += 1\n    return (defeated,)\n'

In [4]:
def cxWholeAritmetic(parent1,parent2,alpha):
    child1 = (alpha*np.array(parent1))+((1-alpha)*np.array(parent2))
    child2 = (alpha*np.array(parent2))+((1-alpha)*np.array(parent1))
    return creator.Individual(child1),creator.Individual(child2)

def cxBlend(parent1,parent2,alpha):
    gamma = (1+2*alpha)*np.random.uniform(size=(len(parent1,))) - alpha
    child1 = (gamma*np.array(parent1))+((1-gamma)*np.array(parent2))
    child2 = (gamma*np.array(parent2))+((1-gamma)*np.array(parent1))
    return creator.Individual(child1),creator.Individual(child2)

In [5]:
def mutGaussian(ind,sigma,indpb):
    mutation = [np.random.normal(0,sigma) if np.random.uniform()<indpb else 0 for i in range(len(ind))]
    return creator.Individual(np.array(ind) + np.array(mutation))

def mutGaussianAdaptive(ind,sigma,tau,indpb):
    new_sigma = sigma * np.exp(np.random.normal(0,tau,size=(len(ind),)))
    new_sigma[new_sigma<0.1] = 0.1
    mutation = [np.random.normal(0,s) if np.random.uniform()<indpb else 0 for s in new_sigma]
    return creator.Individual(np.array(ind) + np.array(mutation))

In [6]:
def best_initializer(pop_size,in_sigma,in_mutpb,in_tau):
    pop = np.random.choice(final_results['solution'],pop_size)
    pop = [creator.Individual(ind) for ind in pop]
    for i in range(len(pop)):
        if np.random.uniform() < in_mutpb:
            pop[i] = mutGaussianAdaptive(pop[i],in_sigma,in_tau,1)
    return pop

In [7]:
def genetic_algorithm(in_sigma,in_tau,in_mutpb,pop_size,n_offspring,tournsize_parent,cxpb,cx_type,
                      alpha,mutpb,mut_type,sigma,indpb,tau,tournsize_survival,ngen,verbose):
    
    global individual_size,env,creator,stats
    
    toolbox = base.Toolbox()
    toolbox.register("weight_bin", np.random.uniform,-0,0)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.weight_bin, n=individual_size)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", fitness_function)

    pop = best_initializer(pop_size,in_sigma,in_mutpb,in_tau)
    hof = {'best_solution':-np.inf,'max_value':-np.inf}

    with multiprocessing.Pool() as pool:
        # evaluate initial population
        for ind, fit in zip(pop, pool.map(toolbox.evaluate,pop)):
            ind.fitness.values = fit

        # record best solution
        for ind in pop:
            if ind.fitness.values[0]>hof['max_value']:
                hof['best_solution'] = ind
                hof['max_value'] = ind.fitness.values[0]

        logbook = tools.Logbook()
        logbook.header = ['gen', 'nevals'] + (stats.fields if stats else []) + ['max_hof']
        record = stats.compile(pop) if stats else {}

        logbook.record(gen=0, nevals=len(pop), **record, max_hof=hof['max_value'])
        if verbose==1:
            print(logbook.stream)

        for g in range(ngen):
            # select parents
            parents = tools.selTournament(individuals=pop,k=n_offspring*pop_size,tournsize=tournsize_parent)

            # create offspring
            offspring = []
            for child1, child2 in zip(parents[::2], parents[1::2]):
                if np.random.uniform() < cxpb:
                    if cx_type=="whole_aritmetic":
                        if alpha=="random":
                            alpha=np.random.uniform()
                            child1,child2 = cxWholeAritmetic(child1, child2, alpha)
                        else:
                            child1,child2 = cxWholeAritmetic(child1, child2, alpha)
                    elif cx_type=="blend":
                        if alpha=="random":
                            alpha=np.random.uniform()
                            child1,child2 = cxBlend(child1, child2, alpha)
                        else:
                            child1,child2 = cxBlend(child1, child2, alpha)
                offspring.append(child1)
                offspring.append(child2)

            # apply mutation to offspring
            for i in range(len(offspring)):
                if np.random.uniform() < mutpb:
                    if mut_type=="normal":
                        if indpb=="random":
                            indpb=np.random.uniform()
                            offspring[i] = mutGaussian(offspring[i],sigma,indpb)
                        else:
                            offspring[i] = mutGaussian(offspring[i],sigma,indpb)
                    elif mut_type=="adaptive":
                        if indpb=="random":
                            indpb=np.random.uniform()
                            offspring[i] = mutGaussianAdaptive(offspring[i],sigma,indpb,tau)
                        else:
                            offspring[i] = mutGaussianAdaptive(offspring[i],sigma,indpb,tau)       

            # evaluate offspring
            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            for ind, fit in zip(invalid_ind, pool.map(toolbox.evaluate,invalid_ind)):
                    ind.fitness.values = fit

            # record best solution
            for ind in offspring:
                if ind.fitness.values[0]>hof['max_value']:
                    hof['best_solution'] = ind
                    hof['max_value'] = ind.fitness.values[0]

            # select the next generation of the population
            pop = tools.selTournament(individuals=offspring,k=pop_size,tournsize=tournsize_survival)

            record = stats.compile(pop) if stats else {}
            logbook.record(gen=g+1, nevals=len(invalid_ind), **record, max_hof=hof['max_value'])
            if verbose==1:
                print(logbook.stream)
        pool.close()
        pool.join()

    return hof,logbook

In [8]:
hidden_neurons = 10
individual_size = parameter_count(hidden_neurons)

headless = True
if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy"

env = Environment(playermode="ai",
                  player_controller=player_controller(hidden_neurons),
                  speed="fastest",
                  enemymode="static",
                  level=2,
                  visuals=False,
                  logs="off")

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("Mean", np.mean)
stats.register("Max", np.max)
stats.register("Std", np.std)

ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5704:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM default


In [9]:
def objective(trial):
    
    params = {'in_sigma':trial.suggest_float("in_sigma", 0.1, 1., step=0.1),
              'in_tau':trial.suggest_float("in_tau", 0.1, 1., step=0.1),
              'in_mutpb':trial.suggest_float("in_mutpb", 0.1, 1., step=0.1),
              'pop_size':trial.suggest_int("pop_size", 50,50,step=50),
              'n_offspring':trial.suggest_int("n_offspring", 1,5,step=1),
              'tournsize_parent':trial.suggest_int("tournsize_parent", 1,10,step=1),
              'cxpb':trial.suggest_float("cxpb", 0.1, 1., step=0.1),
              'cx_type':trial.suggest_categorical("cx_type",["whole_aritmetic","blend"]),
              'alpha':trial.suggest_categorical("alpha",np.arange(0.1,1.1,0.1).tolist()+["random"]),
              'mutpb':trial.suggest_float("mutpb", 0.1, 1., step=0.1),
              'mut_type':trial.suggest_categorical("mut_type",["normal","adaptive"]),
              'sigma':trial.suggest_float("sigma", 0.5, 3, step=0.5),
              'indpb':trial.suggest_categorical("indpb",np.arange(0.1,1.1,0.1).tolist()+["random"]),
              'tau':trial.suggest_float("tau", 0.1, 1., step=0.1),
              'tournsize_survival':trial.suggest_int("tournsize_survival", 1,10,step=1),
              'ngen':trial.suggest_int("ngen", 30,30,step=20),
              'verbose':trial.suggest_int("verbose", 1,1,step=1)}
    
    global hof_tuning
    iteration_logs = []
    for i in range(1):
        hof,logs = genetic_algorithm(**params)
        logs_pd = pd.DataFrame(logs)
        iteration_logs.append(logs_pd)
        hof_tuning.append(hof)
    
    return pd.concat(iteration_logs)['max_hof'].max()


In [10]:
import pickle
with open('/kaggle/input/tune-res-generalist/final_solutions_v2.pickle', 'rb') as handle:
    final_results = pickle.load(handle)

In [11]:
hof_tuning = []

study = optuna.create_study(study_name="GA", direction="maximize")

study.optimize(objective, n_trials=500)

[I 2023-10-15 21:03:01,395] A new study created in memory with name: GA


gen	nevals	Mean  	Max  	Std    	max_hof
0  	50    	80.592	356.8	142.903	356.8  
1  	43    	216.224	276  	79.4073	356.8  
2  	41    	226.1  	276  	125.565	356.8  
3  	39    	266    	276  	68.1393	356.8  
4  	44    	276    	276  	5.68434e-14	356.8  
5  	43    	276    	276  	5.68434e-14	356.8  
6  	47    	257.544	276  	90.4156    	356.8  


Process ForkPoolWorker-3:
Process ForkPoolWorker-1:
Process ForkPoolWorker-4:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target

KeyboardInterrupt: 

In [ ]:
#Trial 16 finished with value: 30.800000000000495 and parameters: {'initial_range': 2, 'pop_size': 50, 'n_offspring': 4, 'tournsize_parent': 6, 'cxpb': 0.6, 'cx_type': 'blend', 'alpha': 0.8, 'mutpb': 0.9, 'mut_type': 'adaptive', 'sigma': 3.0, 'indpb': 0.8, 'tau': 0.30000000000000004, 'tournsize_survival': 7, 'ngen': 30, 'verbose': 1}

In [ ]:
#Trial 0 finished with value: 356.8000000000009 and parameters: {'in_sigma': 0.7000000000000001, 'in_tau': 0.9, 'in_mutpb': 0.6, 'pop_size': 50, 'n_offspring': 4, 'tournsize_parent': 1, 'cxpb': 0.30000000000000004, 'cx_type': 'whole_aritmetic', 'alpha': 'random', 'mutpb': 0.9, 'mut_type': 'normal', 'sigma': 2.5, 'indpb': 0.1, 'tau': 0.8, 'tournsize_survival': 10, 'ngen': 30, 'verbose': 1}

In [15]:
logs_list = []
for i in range(100):
    hof,  logs = genetic_algorithm(**params)
    logs_list.append(hof)

gen	nevals	Mean   	Max  	Std    	max_hof
0  	50    	-77.488	356.8	269.353	356.8  
1  	187   	88.24  	356.8	91.5214	356.8  
2  	189   	124.844	356.8	68.6296	356.8  
3  	189   	141.364	255  	59.6384	356.8  
4  	183   	186.328	255  	61.2149	356.8  
5  	190   	188.664	255  	51.0085	356.8  
6  	186   	202.064	261.2	50.8149	356.8  
7  	191   	204.84 	277.4	53.6337	356.8  
8  	185   	205.268	277.4	58.3902	356.8  
9  	183   	202.964	278.8	67.3876	356.8  
10 	188   	216.152	288.8	59.0886	356.8  
11 	192   	207.628	278.8	61.0066	356.8  
12 	187   	195.888	284.6	62.7244	356.8  
13 	188   	228.596	301.6	60.5974	356.8  
14 	187   	246.764	301.6	65.382 	356.8  
15 	188   	252.956	301.6	54.9084	356.8  
16 	185   	249.084	301.6	63.6657	356.8  
17 	189   	242.688	321  	62.934 	356.8  
18 	180   	279.448	321  	30.6912	356.8  
19 	188   	264.076	321  	49.4524	356.8  
20 	181   	276.476	321  	41.7216	356.8  
21 	181   	281.112	321  	38.6296	356.8  
22 	178   	296.828	321  	16.7656	356.8  
23 	181   	290.6

Process ForkPoolWorker-20:
Process ForkPoolWorker-17:
Process ForkPoolWorker-18:
Process ForkPoolWorker-19:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result =

KeyboardInterrupt: 

In [24]:
with open('best.txt','w') as file:
    for item in son:
        file.write((f"{item}\n"))

In [18]:
son = final_results.loc[15,'solution'].tolist()

In [12]:
params = {'in_sigma': 0.7,
          'in_tau': 0.9,
          'in_mutpb': 0.6,
          'pop_size': 50,
          'n_offspring': 4,
          'tournsize_parent': 1,
          'cxpb': 0.3,
          'cx_type': 'whole_aritmetic',
          'alpha': 'random',
          'mutpb': 0.9,
          'mut_type': 'normal',
          'sigma': 2.5,
          'indpb': 0.1,
          'tau': 0.8,
          'tournsize_survival': 10,
          'ngen': 100,
          'verbose': 1}

In [19]:
son

[-9.705104673283698,
 13.140415789623065,
 16.027693852809975,
 -5.3854654925260075,
 0.13940471776243202,
 -12.506770708317916,
 -0.3808883736058173,
 18.13163218470681,
 -2.6874582613342906,
 4.124070523540349,
 -6.252337270361888,
 -25.074112576498308,
 23.7660509863508,
 19.125883236372815,
 -4.633077095887326,
 -0.46397600798348604,
 -4.705558627586143,
 -12.020440755184255,
 -14.04941173558066,
 6.086766997455086,
 4.79890082517129,
 16.852500893020732,
 18.684162382535078,
 -0.8001015026383058,
 -17.53733788155736,
 -2.756168084205073,
 -11.003906254287298,
 37.357191910014826,
 9.886649621279432,
 28.888313486979015,
 -8.238834709617247,
 28.868551393816254,
 -8.209491082763794,
 -18.648030547910835,
 6.647826476546454,
 -20.205302884262558,
 8.764700507164365,
 -9.002385603576988,
 9.470431107520449,
 10.803540116653043,
 -0.17946730695740165,
 -32.59623607833498,
 -12.575006916995818,
 2.3666947155014,
 9.833315457689245,
 2.4753072347489766,
 5.445952765510237,
 8.1143082857

In [9]:
params = {'initial_range': 2,
          'pop_size': 50,
          'n_offspring': 4,
          'tournsize_parent': 6,
          'cxpb': 0.6,
          'cx_type': 'blend',
          'alpha': 0.8,
          'mutpb': 0.9,
          'mut_type': 'adaptive',
          'sigma': 3.0,
          'indpb': 0.8,
          'tau': 0.3,
          'tournsize_survival': 7,
          'ngen': 500,
          'verbose': 1}

In [15]:
def final_res(sol):
    global env
    res = []
    for enemy in range(1,9):
        env.update_parameter("enemies",[enemy])
        res.append(env.play(sol))
    return res

In [ ]:
"""
params = {'initial_range':,
          'pop_size':,
          'n_offspring':,
          'tournsize_parent':,
          'cxpb':,
          'cx_type':,
          'alpha':,
          'mutpb':,
          'mut_type':,
          'sigma':,
          'indpb':,
          'tau':,
          'tournsize_survival':,
          'ngen':,
          'verbose':}
""""